In [1]:
!nvidia-smi

Mon Jul 11 07:09:16 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   40C    P8    12W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
import os
import glob
import cv2 
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from keras.utils.np_utils import to_categorical
from keras.models import load_model  
from sklearn.model_selection import train_test_split
from keras import optimizers
import seaborn as sns
import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Dense, Flatten ,Input, Conv2D ,MaxPooling2D ,Dropout ,BatchNormalization ,GlobalMaxPool2D
from tensorflow.keras.models import Sequential , Model , load_model
from tensorflow.keras.optimizers import Adam, RMSprop
from tensorflow.keras.utils import plot_model
from tensorflow.keras.callbacks import ModelCheckpoint
from skimage.feature import greycomatrix, greycoprops
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import load_img, img_to_array, array_to_img
import matplotlib.image as mpimg
import pickle
from keras.utils import np_utils
from sklearn.preprocessing import LabelEncoder
from keras.utils.np_utils import to_categorical
from tensorflow.keras.optimizers import RMSprop,Adam
from tensorflow import keras 
from tensorflow.keras import layers
from keras.models import Model
from sklearn import model_selection
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score,classification_report,confusion_matrix

In [5]:
#HYPERPARAMETERS
batch_size = 16
num_classes = 2
img_dim = 128
n_channel = 3
input_shape = (img_dim,img_dim,n_channel)

In [6]:
#Creating dataframe of (Image,Label) from the images
def createFrame(path,IMG_DIM):
    train_imgs = []
    labels = []
    #getting all folder name
    directories = os.listdir(path)
    for i in range(len(directories)):
        ls = []
        temp = []
        curPath = path +'/' +directories[i] + '/*'
        #getting all files name
        ls = glob.glob(curPath)
        for img in ls:
            x = img_to_array(load_img(img,target_size = IMG_DIM))
            temp.append(x)

        #print(len(ls))
        train_imgs  = train_imgs + temp
        label = []
        label = [i]*len(ls)
        labels += label

    df = pd.DataFrame(list(zip(train_imgs,labels)))
    df = df.sample(frac = 1) 
    return df

In [7]:
def data_split(df):
    df_imgs = df[0]
    df_imgs = list(df[0])
    df_imgs = np.array(df_imgs)
    df_imgs = df_imgs/255
    
    df_labels = df[1]
    df_labels = np.array(df[1])
    encoder = LabelEncoder()
    encoder.fit(df_labels)
    df_labels = encoder.transform(df_labels)
    df_labels = np_utils.to_categorical(df_labels)

    return df_imgs, df_labels


In [8]:
train_dir = '/content/drive/MyDrive/cxr_splitdata/train'
val_dir = '/content/drive/MyDrive/cxr_splitdata/val'
test_dir = '/content/drive/MyDrive/cxr_splitdata/test'

In [9]:
df_train = createFrame(train_dir,input_shape)

In [10]:
X_train, y_train = data_split(df_train)

In [11]:
df_val = createFrame(val_dir,input_shape)

In [12]:
X_val, y_val = data_split(df_val)

In [13]:
df_test = createFrame(test_dir,input_shape)

In [14]:
X_test, y_test = data_split(df_test)

In [15]:
print(f'X_train shape, y_train shape: {X_train.shape, y_train.shape}')
print(f'X_val shape, y_val shape: {X_val.shape, y_val.shape}')
print(f'X_test shape, y_test shape: {X_test.shape, y_test.shape}')

X_train shape, y_train shape: ((5845, 128, 128, 3), (5845, 2))
X_val shape, y_val shape: ((730, 128, 128, 3), (730, 2))
X_test shape, y_test shape: ((733, 128, 128, 3), (733, 2))


In [18]:
num_epochs = 300

In [25]:
#RESNET MODEL

#num_classes= 2
def Resnet():
    model_res =  keras.applications.ResNet50(input_shape = input_shape, include_top = False, weights = 'imagenet')
    for layer in model_res.layers:
        layer.trainable = False

    last = model_res.output

    x = Flatten()(last)
    x = Dense(512, activation = 'relu')(x)
    x = Dense(256, activation = 'relu')(x)
    x = Dropout(rate = 0.30)(x)
    x = Dense(64, activation = 'relu')(x)
    x = Dense(2, activation = 'sigmoid')(x)

    #Compiling model
    model = Model(inputs = model_res.input, outputs = x)
    #model.summary()

    opt1 = Adam(learning_rate=0.00003)
    resnet_checkpointing = ModelCheckpoint("resnet_best.h5", monitor='val_accuracy', verbose=1, mode='max', save_best_only=True)
    model.compile(optimizer = opt1 , loss = 'binary_crossentropy', metrics = ['accuracy'])
    history = model.fit(X_train, y_train, batch_size=batch_size, epochs = num_epochs, validation_data = (X_val, y_val), callbacks=[resnet_checkpointing])

    print('---------------MODEL TRAINED-----------------')

    res_saved = tf.keras.models.load_model('./resnet_best.h5')
    
    return res_saved


In [26]:
#VGG16 MODEL

def Vgg16():
    #from tensorflow.keras.applications import VGG16
    model_vgg = keras.applications.VGG16(input_shape = input_shape, 
                                include_top = False, 
                                weights = 'imagenet')
    for layer in model_vgg.layers:
        layer.trainable = False

    #Getting desired layer output
    last_layer = model_vgg.get_layer('block5_pool')
    last = last_layer.output

    x = Flatten()(last)
    x = Dense(512, activation = 'relu')(x)
    x = Dense(256, activation = 'relu')(x)
    x = Dropout(0.30)(x)
    x = Dense(64, activation = 'relu')(x)
    x = Dense(2, activation = 'sigmoid')(x)

    #Compiling model
    model = Model(inputs = model_vgg.input, outputs = x)
    opt1 = Adam(learning_rate=0.00003)
    vgg16_checkpointing = ModelCheckpoint("vgg16_best.h5", monitor='val_accuracy', verbose=1, mode='max', save_best_only=True)
    model.compile(optimizer = opt1 , loss = 'binary_crossentropy', metrics = ['accuracy'])
    history = model.fit(X_train, y_train, batch_size=batch_size, epochs = num_epochs, validation_data = (X_val, y_val), callbacks=[vgg16_checkpointing])

    print('---------------MODEL TRAINED-----------------')
    
    vgg_saved = tf.keras.models.load_model('./vgg16_best.h5')

    return vgg_saved



In [27]:
res_model_best = Resnet() 

Epoch 1/300
365/366 [============================>.] - ETA: 0s - loss: 0.6927 - accuracy: 0.5315
Epoch 1: val_accuracy improved from -inf to 0.50274, saving model to resnet_best.h5
366/366 [==============================] - 15s 32ms/step - loss: 0.6927 - accuracy: 0.5314 - val_loss: 0.6830 - val_accuracy: 0.5027
Epoch 2/300
365/366 [============================>.] - ETA: 0s - loss: 0.6754 - accuracy: 0.5805
Epoch 2: val_accuracy improved from 0.50274 to 0.59589, saving model to resnet_best.h5
366/366 [==============================] - 11s 30ms/step - loss: 0.6754 - accuracy: 0.5803 - val_loss: 0.6498 - val_accuracy: 0.5959
Epoch 3/300
365/366 [============================>.] - ETA: 0s - loss: 0.6422 - accuracy: 0.6462
Epoch 3: val_accuracy improved from 0.59589 to 0.74384, saving model to resnet_best.h5
366/366 [==============================] - 11s 30ms/step - loss: 0.6421 - accuracy: 0.6464 - val_loss: 0.5931 - val_accuracy: 0.7438
Epoch 4/300
364/366 [============================>.]

In [28]:
vgg_model_best = Vgg16()

58900480/58889256 [==============================] - 0s 0us/step
Epoch 1/300
366/366 [==============================] - ETA: 0s - loss: 0.4442 - accuracy: 0.8062
Epoch 1: val_accuracy improved from -inf to 0.85479, saving model to vgg16_best.h5
366/366 [==============================] - 16s 39ms/step - loss: 0.4442 - accuracy: 0.8062 - val_loss: 0.3567 - val_accuracy: 0.8548
Epoch 2/300
365/366 [============================>.] - ETA: 0s - loss: 0.3183 - accuracy: 0.8630
Epoch 2: val_accuracy improved from 0.85479 to 0.89041, saving model to vgg16_best.h5
366/366 [==============================] - 13s 34ms/step - loss: 0.3183 - accuracy: 0.8630 - val_loss: 0.2850 - val_accuracy: 0.8904
Epoch 3/300
365/366 [============================>.] - ETA: 0s - loss: 0.2716 - accuracy: 0.8872
Epoch 3: val_accuracy improved from 0.89041 to 0.90137, saving model to vgg16_best.h5
366/366 [==============================] - 13s 35ms/step - loss: 0.2716 - accuracy: 0.8873 - val_loss: 0.2483 - val_accurac

In [ ]:
#res_saved = tf.keras.models.load_model('./resnet_best.h5')
#vgg_saved = tf.keras.models.load_model('./vgg16_best.h5')

In [29]:
print(f'Resnet model accuracy on test datset : { (res_model_best.evaluate(X_test,y_test)) }')
print(f'Vgg16 model accuracy on test dataset : { (vgg_model_best.evaluate(X_test,y_test)) }')

23/23 [==============================] - 3s 61ms/step - loss: 0.3246 - accuracy: 0.8581
Resnet model accuracy on test datset : [0.324551522731781, 0.8581173419952393]
23/23 [==============================] - 4s 118ms/step - loss: 0.2040 - accuracy: 0.9454
Vgg16 model accuracy on test dataset : [0.2040153443813324, 0.9454297423362732]


In [30]:
def stack_model(models):
    input = Input(shape=input_shape)
    result = []

    for model in models:
        model._name = model.get_layer(index=0)._name + "-test" + str(models.index(model)+1)
        result.append(model(input))

    stackmodel = tf.keras.layers.concatenate(result,axis=-1)
    stackmodel = Dense(64, activation='relu')(stackmodel)
    stackmodel = Dropout(0.1)(stackmodel)
    stackmodel = Dense(2, activation='sigmoid')(stackmodel)
    stackmodel_main = Model(input,stackmodel)

    opt_stack = Adam(lr=5e-5)
    
    stacked_checkpoint = ModelCheckpoint("stackmodel.h5", monitor='val_accuracy', verbose=1, mode='max', save_best_only=True)
    stackmodel_main.compile(optimizer= opt_stack, loss='binary_crossentropy', metrics=['accuracy'])

    stacked_hist = stackmodel_main.fit(X_train,y_train, batch_size=batch_size, epochs=num_epochs, validation_data= (X_val,y_val), verbose=1, callbacks=[stacked_checkpoint])
    
    print('-----------------STACK MODEL TRAINED---------------')

    stack_saved = tf.keras.models.load_model("./stackmodel.h5")

    return stack_saved


In [31]:
models =[res_model_best, vgg_model_best]

In [32]:
stacked_model_best = stack_model(models)

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


Epoch 1/300
366/366 [==============================] - ETA: 0s - loss: 0.5879 - accuracy: 0.9629
Epoch 1: val_accuracy improved from -inf to 0.93836, saving model to stackmodel.h5
366/366 [==============================] - 28s 64ms/step - loss: 0.5879 - accuracy: 0.9629 - val_loss: 0.5691 - val_accuracy: 0.9384
Epoch 2/300
366/366 [==============================] - ETA: 0s - loss: 0.5224 - accuracy: 0.9860
Epoch 2: val_accuracy improved from 0.93836 to 0.94247, saving model to stackmodel.h5
366/366 [==============================] - 21s 56ms/step - loss: 0.5224 - accuracy: 0.9860 - val_loss: 0.5127 - val_accuracy: 0.9425
Epoch 3/300
366/366 [==============================] - ETA: 0s - loss: 0.4710 - accuracy: 0.9692
Epoch 3: val_accuracy did not improve from 0.94247
366/366 [==============================] - 19s 53ms/step - loss: 0.4710 - accuracy: 0.9692 - val_loss: 0.4632 - val_accuracy: 0.9356
Epoch 4/300
365/366 [============================>.] - ETA: 0s - loss: 0.3969 - accuracy: 

In [33]:
print(f'Stacked model accuracy : {stacked_model_best.evaluate(X_test, y_test)}')

23/23 [==============================] - 8s 191ms/step - loss: 0.4566 - accuracy: 0.9563
Stacked model accuracy : [0.456570565700531, 0.9563437700271606]
